Damn, I just lost a shit ton of resources.


# Goals: 
1. Find how Shepard did his experimentalcardeasefactor
2. Try to implement a small-scale prediction model that can scrape data from the history
3. Try to implement Tabibian's model, and make it update all of the cards. 

Add a button that updates all the cards following Tabibian's model. 

# REPL Testing
- Make changes, restart Anki


# Resources
Repos
- https://github.com/eshapard/experimentalCardEaseFactor/
- https://github.com/duolingo/halflife-regression
- https://github.com/Networks-Learning/memorize
- https://github.com/dae/anki

Anki
- https://github.com/ankidroid/Anki-Android/wiki/Database-Structure
- The debug console is cmd ;


MEMORIZE Resources
- https://www.reddit.com/r/Anki/comments/cghyiy/optimized_spaced_repetition_algorithm/ (Utkarsh reponds)
- http://learning.mpi-sws.org/memorize/

Making an Addon
- https://www.juliensobczak.com/write/2016/12/26/anki-scripting.html

Paying
- https://www.reddit.com/r/Anki/comments/e0dazp/how_does_the_anki_community_feel_about_paying_for/




# Right now, try to figure out a way to use Tabibian's model on my own dataset. 
## Section 1: Importing Stuff

There can be problems with importing mw from aqt. I had to pip install PyQt5 and PyQtWebEngine.


Also, based on https://bugs.launchpad.net/ubuntu/+source/anki/+bug/1825722, I had to go into anki/aqt/qt.py and change 

-from PyQt5.QtWebEngineWidgets import QWebEnginePage
+from PyQt5.QtWebEngineWidgets import *
try:
from PyQt5 import sip
except ImportError:

---
1. git clone the anki repo
2. get commandlinetools
3. run build.sh following instructions from https://github.com/dae/anki/blob/master/README.development
4. install requirements
4. brew install python mplayer lame portaudio




In [32]:
import json
import os
import pandas as pd
import sqlite3
import argparse
import csv
import gzip
import math
import random
import sys
from sys import intern
from collections import defaultdict, namedtuple
from sklearn.metrics import roc_auc_score


curdir = os.getcwd()
os.chdir("/Users/jacob/anki")

from anki import Collection
from anki.utils import intTime
import time

from aqt import mw
from aqt.reviewer import Reviewer
from aqt.utils import showInfo
from anki.hooks import wrap, addHook, runHook
from aqt.qt import *


os.chdir(curdir)

In [17]:
collection_path = "/Users/jacob/Library/Application Support/Anki2/Minerva/collection.anki2"
col = Collection(collection_path)
print(col.db)
col.close()

## Settles and Meeder's Spaced Repetition Model (Tabibian's Version)

### Functions to do Data analysis WITHOUT using Collection

In [31]:
collection_path = "/Users/jacob/Library/Application Support/Anki2/Minerva/collection.anki2"

def totalcards(collection_path):
    """
    input: the path of the collection
    output: the number of total cards.
    """
    col = Collection(collection_path)
    query = "SELECT COUNT(id) from cards"
    query_result = col.db.scalar(query)
    col.close()
    return (f'There are {query_result} total cards')

def duestatus(collection_path, hrs_ahead = 24):
    """
    input: the path of the collection, the integer time
    """
    col = Collection(collection_path)
    next_hrs_ahead = intTime() +  hrs_ahead * 60 * 60 # today in int time plus x hrs * 60 mins * 60 seconds

    cardsduedict = {"schedburied": col.db.scalar(f"select count(id) from cards where queue = -3 and due < {next_hrs_ahead}"),
                    "userburied": col.db.scalar(f"select count(id) from cards where queue = -2 and due < {next_hrs_ahead}"),
                    "suspended": col.db.scalar(f"select count(id) from cards where queue = -1 and due < {next_hrs_ahead}"),
                    "new": col.db.scalar(f"select count(id) from cards where queue = 0 and due < {next_hrs_ahead}"),
                    "learning": col.db.scalar(f"select count(id) from cards where queue = 1 and due < {next_hrs_ahead}"),
                    "due": col.db.scalar(f"select count(id) from cards where queue = 2 and due < {next_hrs_ahead}"),
                    "next rev at least a day": col.db.scalar(f"select count(id) from cards where queue = 3 and due < {next_hrs_ahead}")}
    

    print(f'In the next {hrs_ahead} hours ahead, cards of the following type are due:')

    col.close()
    return cardsduedict

def getnextcardinfo(collection_path, cardid = 1557121425301):
    """
    input: a collection path and a cardid.
    output: general information about a card, including its nid, type, queue, due, ivl, reps and lapses
    """
    
    
    col = Collection(collection_path)
#     print(col.db.list(f"select type, queue, due, ivl, reps, lapses from cards where queue = 1"))
    nid, types, queue, due, ivl, reps, lapses = col.db.first(f"select nid, type, queue, due, ivl, reps, lapses from cards where id = {cardid}")
    fields = col.db.all(f"select flds from notes where id = {nid}")
    
    text = col.db.all(f"select flds from notes where id = {nid}")[0]
    typedict = {0:'new', 1:'learning', 2:'due'}
    queuedict = {-3: 'schedburied', -2:'userburied', -1:'suspended', 0:'new', 1:'learning', 2:'due', 3:'in learning +1'}
    ivleval = lambda x: f"{-x} seconds" if x < 0 else f"{x} days"
    
    col.close()

    return (f"For card id {cardid}, type is '{typedict[types]}', queue status is '{queuedict[queue]}', interval is '{ivleval(ivl)}', field is {fields}")


print(totalcards(collection_path))
print()
print(getnextcardinfo(collection_path))
print()
print(duestatus(collection_path))


There are 3752 total cards

For card id 1557121425301, type is 'learning', queue status is 'learning', interval is '0 days', field is [('How does Alvarez &amp; Fournier (2012) define brand flings?\x1fThey are hedonic, noncommited, time-bounded relationships driven by impulse and the need to experiment, and associated with post-termination feelings of shame and regret.&nbsp;\x1f',)]

In the next 24 hours ahead, cards of the following type are due:
{'schedburied': 0, 'userburied': 0, 'suspended': 12, 'new': 2291, 'learning': 53, 'due': 1365, 'next rev at least a day': 31}
